In [29]:
import numpy as np
from scipy.constants import point

from image_processing import *
import cv2
import tqdm
import os

PATH = "floor_1.pdf"
Image.MAX_IMAGE_PIXELS = None

In [30]:
opencv_image = floor_plan_to_opencv_image(PATH)

opencv_image.shape

(17542, 16542, 3)

In [31]:
def compress(image, factor=10):
    height, width = image.shape
    scaled_height = height // factor
    scaled_width = width // factor
    result = np.zeros((scaled_height, scaled_width))

    for result_i in tqdm.trange(scaled_height):

        # # once again, sliding window
        # # initial
        # current = 0
        # 
        # for original_i in range(result_i * factor, min((result_i + 1) * (factor + 1), height)):
        #         for original_j in range(min(result_j * (factor + 1), width)):
        #             current += image[original_i, original_j]

        for result_j in range(scaled_width):
            # print(result_i * factor, min((result_i + 1) * factor, height))
            # print(result_j * factor, min((result_j + 1) * factor, width))
            cnt = 0
            for original_i in range(result_i * factor, min((result_i + 1) * factor, height)):
                for original_j in range(result_j * factor, min((result_j + 1) * factor, width)):
                    cnt += (image[original_i, original_j] == 0)
            result[result_i, result_j] = 255 - (255 * cnt) // (factor * factor)

    return result

In [32]:
# scaled_image = compress(main_image)
# cv.imwrite("IMAGES/scaled.png", scaled_image)

In [33]:
# def process_image(img):
#     # Convert image to grayscale
#     # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     
#     # Apply a slight Gaussian blur to reduce noise
#     blurred = cv2.GaussianBlur(img, (3, 3), 0)
#     
#     # Invert the image (so lines are white on black background)
#     inverted = cv2.bitwise_not(blurred)
#     
#     # Calculate local density using a moving window
#     window_size = 15  # Adjust this size for better results
#     kernel = np.ones((window_size, window_size), np.uint8)
#     density_map = cv2.filter2D(inverted, -1, kernel)
#     
#     # Normalize the density map for better contrast
#     normalized = cv2.normalize(density_map, None, 0, 255, cv2.NORM_MINMAX)
#     
#     # Convert to uint8 format for display
#     normalized = np.uint8(normalized)
#     
#     # inverted = cv.bitwise_not(normalized)
#     
#     return inverted

In [34]:
# scaled_image = process_image(main_image)
# 
# cv.imwrite("IMAGES/scaled.png", scaled_image)

In [35]:
def transform_architectural_drawing(img, output_path):
    """
    Transform an architectural drawing to a higher contrast, simplified version
    
    Parameters:
    image_path (str): Path to the input image
    output_path (str): Path to save the processed image
    """
    # Read the image in grayscale

    if img is None:
        raise ValueError("Could not load the image")

    # Apply preprocessing steps
    # 1. Denoise the image
    denoised = cv2.fastNlMeansDenoising(img)

    # 2. Increase contrast using adaptive histogram equalization
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    contrast_enhanced = clahe.apply(denoised)

    # 3. Apply threshold to make it more binary-like
    _, binary = cv2.threshold(contrast_enhanced, 200, 255, cv2.THRESH_BINARY)

    # 4. Clean up small noise using morphological operations
    kernel = np.ones((2, 2), np.uint8)
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    # 5. Enhance lines
    edges = cv2.Canny(cleaned, 50, 150)
    dilated_edges = cv2.dilate(edges, kernel, iterations=1)

    # 6. Combine the binary image with enhanced edges
    result = cv2.bitwise_and(cleaned, cleaned, mask=cv2.bitwise_not(dilated_edges))

    # Save the result
    cv2.imwrite(output_path, result)

    return result

In [36]:
# def get_neighborhood(i, sz, bound):
#     return i, min(i + sz, bound)
# 
# def non_linear_filter(image, kernel_size=3, fraction=2):
#     # Get the height and width of the image
#     height, width = image.shape
# 
#     # Create an output image initialized to zeros
#     output_image = np.zeros_like(image)
# 
#     # Iterate over each pixel in the original image
#     for i in tqdm.trange(height):
#         # implement sliding window
# 
#         # initial state
#         # current_black = 0
#         # for ik in range(*get_neighborhood(i, kernel_size, height)):
#         #     for jk in range(*get_neighborhood(0, kernel_size, width)):
#         #         if image[ik, jk] == 0:
#         #             current_black += 1
# 
#         for j in range(1, width):
#             # Extract the kernel window
#             # cross does not work that well as there are diagonal sections
#             kernel_window = []
#             # what if we try give more weight to diagonal thingies?
#             for ik in range(*get_neighborhood(i, kernel_size, height)):
#                 for jk in range(*get_neighborhood(j, kernel_size, width)):
#                     multiplier = abs(ik - i) / max(1, abs(jk - j)) + 1
#                     # if abs(ik - i) == abs(jk - j): # on diagonal
#                     #     multiplier = 2
#                     kernel_window.append((image[ik, jk], multiplier))
# 
# 
#             # Count the number of black pixels in the window
#             current_black = sum(map(lambda x: x[1] if x[0] == 0 else 0, kernel_window))
# 
#             # Determine if more than half the pixels are black
#             threshold = len(kernel_window) / fraction
# 
#             # for ik in range(*get_neighborhood(i, kernel_size, height)):
#             #     # subtract previous
#             #     if image[ik, j - 1] == 0:
#             #         current_black -= 1
#             #     # add current
#             #     if j + kernel_size - 1 < width and image[ik, j + kernel_size - 1] == 0:
#             #         current_black += 1
#             # 
#             # threshold = kernel_size * kernel_size / fraction  # incorrect for edge cases, but whatever
#             output_image[i, j] = 0 if current_black > threshold else 255
# 
#     return output_image
# 
# 
# def soft_max_kernel(image, kernel_size, fraction, f=False):
#     filename = f"IMAGES/default-test-identify{kernel_size}-{fraction}.png"
#     if os.path.isfile(filename) and not f:
#         print("warning! file already exists! pass f flag if you want to overwrite it")
#         return False, None
#     scanned = non_linear_filter(image, kernel_size, fraction)
#     return cv.imwrite(filename, scanned), scanned

In [37]:
# f, res = soft_max_kernel(main_image, 10, 10, True)
# f

In [57]:
no_details = remove_page_details(opencv_image, eps=10, min_samples=30)
no_details.shape

100%|██████████| 3508/3508 [00:01<00:00, 2291.25it/s]


446
[(np.int64(6), 174469), (np.int64(-1), 42567), (np.int64(23), 7695), (np.int64(66), 727), (np.int64(223), 595), (np.int64(0), 469), (np.int64(169), 443), (np.int64(414), 395), (np.int64(433), 387), (np.int64(41), 350)]


(3508, 3308)

In [58]:
# def get_neighborhood(i, sz, bound):
#     return i, min(i + sz, bound)
# 
# def non_linear_filter(image, kernel_size=3, fraction=2):
#     # Get the height and width of the image
#     height, width = image.shape
# 
#     # Create an output image initialized to zeros
#     output_image = np.zeros_like(image)
# 
#     # Iterate over each pixel in the original image
#     for i in tqdm.trange(height):
#         # implement sliding window
# 
#         # initial state
#         # current_black = 0
#         # for ik in range(*get_neighborhood(i, kernel_size, height)):
#         #     for jk in range(*get_neighborhood(0, kernel_size, width)):
#         #         if image[ik, jk] == 0:
#         #             current_black += 1
# 
#         for j in range(1, width):
#             # Extract the kernel window
#             # cross does not work that well as there are diagonal sections
#             kernel_window = []
#             # what if we try give more weight to diagonal thingies?
#             for ik in range(*get_neighborhood(i, kernel_size, height)):
#                 for jk in range(*get_neighborhood(j, kernel_size, width)):
#                     # multiplier = abs(ik - i) / max(1, abs(jk - j)) + 1
#                     # if abs(ik - i) == abs(jk - j): # on diagonal
#                     #     multiplier = 2
#                     multiplier = 1
#                     kernel_window.append((image[ik, jk], multiplier))
# 
# 
#             # Count the number of black pixels in the window
#             current_black = sum(map(lambda x: x[1] if x[0] == 0 else 0, kernel_window))
# 
#             # Determine if more than half the pixels are black
#             threshold = len(kernel_window) / fraction
# 
#             # for ik in range(*get_neighborhood(i, kernel_size, height)):
#             #     # subtract previous
#             #     if image[ik, j - 1] == 0:
#             #         current_black -= 1
#             #     # add current
#             #     if j + kernel_size - 1 < width and image[ik, j + kernel_size - 1] == 0:
#             #         current_black += 1
#             # 
#             # threshold = kernel_size * kernel_size / fraction  # incorrect for edge cases, but whatever
#             output_image[i, j] = 0 if current_black > threshold else 255
# 
#     return output_image
# 
# 
# def soft_max_kernel(image, kernel_size, fraction, f=False):
#     filename = f"IMAGES/post-clustering-softmax{kernel_size}-{fraction}.png"
#     if os.path.isfile(filename) and not f:
#         print("warning! file already exists! pass f flag if you want to overwrite it")
#         return False, None
#     scanned = non_linear_filter(image, kernel_size, fraction)
#     return cv.imwrite(filename, scanned), scanned

In [59]:
# f, res = soft_max_kernel(no_details, 5, 3, True)
# f

In [60]:
def extend_line(line, height, width):
    p1 = np.array(line[0:2])
    p2 = np.array(line[2:4])

    if (p2 - p1)[0] == 0:
        p1[1] = 0
        p2[1] = height
    else:
        k = (p2[1] - p1[1]) / (p2[0] - p1[0])
        b = p2[1] - p2[0] * k

        p1[0] = 0
        p1[1] = b

        p2[0] = width
        p2[1] = k * p2[0] + b
    return *p1, *p2

In [61]:
# try to find lines
def find_lines(image):
    HALF_LENGTH = 1000
    segments = []
    points = []
    output = cv.cvtColor(image, cv.COLOR_GRAY2BGR)  # assumes image in grayscale
    # edges = cv.Canny(image, 1, 50, None, 3)
    # lines = cv.HoughLines(edges, 1, np.pi / 180, threshold=200)
    # # return lines
    # for line in lines:
    #     rho = line[0][0]
    #     theta = line[0][1]
    #     a = np.cos(theta)
    #     b = np.sin(theta)
    #     x0 = a * rho
    #     y0 = b * rho
    #     pt1 = (int(x0 + HALF_LENGTH * (-b)), int(y0 + HALF_LENGTH * a))
    #     pt2 = (int(x0 - HALF_LENGTH * (-b)), int(y0 - HALF_LENGTH * a))
    #     cv.line(output, pt1, pt2, (0, 0, 255), 1, cv.LINE_AA)
    h, w = image.shape
    edges = cv.Canny(image, 50, 150, apertureSize=3)
    lines = cv.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=10)
    for line in lines:
        # x1, y1, x2, y2 = extend_line(line[0], h, w)
        x1, y1, x2, y2 = line[0]
        # segments.append(line[0])
        points.append(np.array((x1, y1)))
        points.append(np.array((x2, y2)))
        segments.append(line[0])
        cv.line(output, (x1, y1), (x2, y2), (0, 0, 255), 1)
    return points, segments, output

In [62]:
pts, segs, lines_no_details = find_lines(no_details.astype(np.uint8))
print(lines_no_details.shape)
cv.imwrite("IMAGES/lines_no_details.png", lines_no_details)

(3508, 3308, 3)


True

In [63]:
pts

[array([ 912, 3348], dtype=int32),
 array([ 912, 1604], dtype=int32),
 array([1764, 2437], dtype=int32),
 array([1764, 1000], dtype=int32),
 array([1769, 3416], dtype=int32),
 array([1769, 3192], dtype=int32),
 array([1780, 2810], dtype=int32),
 array([1780, 2477], dtype=int32),
 array([1755, 2108], dtype=int32),
 array([1755, 1654], dtype=int32),
 array([1778, 2121], dtype=int32),
 array([1778, 1642], dtype=int32),
 array([1753, 2108], dtype=int32),
 array([1753, 1654], dtype=int32),
 array([1766, 2796], dtype=int32),
 array([1766, 2478], dtype=int32),
 array([ 930, 2674], dtype=int32),
 array([ 930, 1571], dtype=int32),
 array([1771, 1417], dtype=int32),
 array([1771, 1000], dtype=int32),
 array([ 928, 3195], dtype=int32),
 array([ 928, 3121], dtype=int32),
 array([ 674, 3346], dtype=int32),
 array([1797, 3346], dtype=int32),
 array([1622, 1246], dtype=int32),
 array([1622, 1016], dtype=int32),
 array([ 675, 3348], dtype=int32),
 array([1742, 3348], dtype=int32),
 array([1624, 2064],

In [64]:
len(pts)

1948

In [65]:
output_with_pts = cv.cvtColor(no_details.astype(np.uint8).copy(), cv.COLOR_GRAY2BGR)

for pt in pts:
    cv.circle(output_with_pts, (int(pt[0]), int(pt[1])), 10, (0, 0, 255), -1)
cv.imwrite("IMAGES/pts.png", output_with_pts)

True

In [66]:
def equal_pts(p1, p2):
    return p1[0] == p2[0] and p1[1] == p2[1]

to_delete = []
for pt in pts:
    # current = None
    close = 0
    for pt2 in pts:
        if equal_pts(pt, pt2):
            continue
        if np.linalg.norm(pt - pt2) < 50:
            close += 1
    if close < 3:
        to_delete.append(pt)

print(to_delete)
for pt in to_delete:
    pts = list(filter(lambda p: not equal_pts(p, pt), pts))
            

[array([1769, 3416], dtype=int32), array([ 674, 3346], dtype=int32), array([1622, 1246], dtype=int32), array([ 675, 3348], dtype=int32), array([1624, 1700], dtype=int32), array([ 101, 3235], dtype=int32), array([ 675, 2774], dtype=int32), array([ 829, 1739], dtype=int32), array([ 831, 2398], dtype=int32), array([1885, 2350], dtype=int32), array([1885, 2092], dtype=int32), array([ 831, 1727], dtype=int32), array([ 125, 2353], dtype=int32), array([ 814, 2171], dtype=int32), array([ 816, 2173], dtype=int32), array([ 123, 2351], dtype=int32), array([ 819, 1735], dtype=int32), array([ 124, 2781], dtype=int32), array([ 103, 3237], dtype=int32), array([ 700, 2726], dtype=int32), array([ 205, 2690], dtype=int32), array([ 207, 2692], dtype=int32), array([ 314, 2571], dtype=int32), array([ 205, 3125], dtype=int32), array([ 727, 2272], dtype=int32), array([ 126, 2783], dtype=int32), array([ 626, 2379], dtype=int32), array([ 303, 2151], dtype=int32), array([ 203, 3123], dtype=int32), array([ 628, 

In [67]:
len(pts)

1678

In [68]:
def determinant(a11, a12, a21, a22):
    return a11 * a22 - a21 * a12


def get_abc(line):  # assumes numpy
    p1 = np.array(line[0:2], dtype=np.int64)
    p2 = np.array(line[2:4], dtype=np.int64)
    a = p1[1] - p2[1]
    b = p2[0] - p1[0]
    c = - a * p1[0] - b * p1[1]
    return a, b, c


def line_intersection(line1, line2):
    a1, b1, c1 = get_abc(line1)
    a2, b2, c2 = get_abc(line2)

    det = determinant(a1, b1, a2, b2)
    if det == 0: # ignore equal
        return None

    return np.array((
        determinant(-c1, b1, -c2, b2), 
        determinant(a1, -c1, a2, -c2)
    )) / det


def point_in_segment(p, seg):
    # assumes that p is on the line
    return (
            min(seg[0], seg[2]) <= p[0] <= max(seg[0], seg[2])
    and
            min(seg[1], seg[3]) <= p[1] <= max(seg[1], seg[3])
    )

def segment_intersection(seg1, seg2):
    # find line intersection, check that it is inside both segments
    potential = line_intersection(seg1, seg2)
    # print(potential)
    if potential is None:
        return None
    
    # if potential[1] < 0:
    #     print(seg1, seg2, potential)
    #     print()
    #     print()
    #     print()
    #     return
    if not point_in_segment(potential, seg1) or not point_in_segment(potential, seg2):
        return None
    return potential

In [69]:
# points = []
# # for i in range(len(found_segs)):
# #     for j in range(len(found_segs)):
# #         if i == j:
# #             continue
# #         intersection = segment_intersection(found_segs[i], found_segs[j])
# #         if intersection is not None:
# #             points.append(intersection)
# 
# for s in found_segs:
#     points.append(s[:2])
#     points.append(s[2:4])

In [70]:
# pts = np.sort(pts, 0)
# list(pts)

In [71]:
from concave_hull import concave_hull, concave_hull_indexes

In [72]:
concave = concave_hull(pts, concavity=3)

In [73]:
# list(np.sort(concave, 0))

In [74]:
len(pts), len(concave)

(1678, 207)

In [75]:
output_with_concave = cv.cvtColor(no_details.astype(np.uint8).copy(), cv.COLOR_GRAY2BGR)

for i in range(len(concave)):
    # print(i, j)
    cv.line(output_with_concave, concave[i - 1], concave[i], (0, 0, 255), 3)
    cv.circle(output_with_concave, concave[i - 1], 10, (255, 0, 0), -1)
    cv.circle(output_with_concave, concave[i], 10, (255, 0, 0), -1)
    # output_with_concave[j][i] = (0, 0, 255) # cause x and y

cv.imwrite("IMAGES/concave.png", output_with_concave)

True